# 3.1 Assignment and Local State

## 3.1.1 Local State Variables

### Exercise 3.1
An _accumulator_ is a procedure that is called repeatedly with a single numeric argument and accumulates its arguments into a sum. Each time it is called, it returns the currently accumulated sum. Write a procedure `make-accumulator` that generates accumulators, each maintaining an independent sum. The input to `make-accumulator` should specify the initial value of the sum; for example

```scheme
(define A (make-accumulator 5)) (A 10)
15
(A 10)
25
```

----

In [3]:
(define (make-accumulator x)
  (lambda (y) 
    (begin
      (set! x (+ x y))
      x)))

;; Validate
(define A (make-accumulator 5))
(A 10)

15

### Exercise 3.2
In software-testing applications, it is useful to be able to count the number of times a given procedure is called during the course of a computation. Write a procedure `make-monitored` that takes as input a procedure, `f`, that itself takes one input. The result returned by `make- monitored` is a third procedure, say `mf`, that keeps track of the number of times it has been called by maintaining an internal counter. If the input to `mf` is the special symbol `how-many-calls?`, then `mf` returns the value of the counter. If the input is the special symbol `reset-count`, then `mf` re- sets the counter to zero. For any other input, `mf` returns the result of calling `f` on that input and increments the counter. For instance, we could make a monitored version of the sqrt procedure:

```scheme
(define s (make-monitored sqrt)) (s 100)
10
(s 'how-many-calls?)
1
```

----

In [6]:
(define (make-monitored f)
  (let ([times 0])
    (lambda (in) 
      (cond [(eq? in 'how-many-calls?) times]
            [(eq? in 'reset-count) (set! times 0)]
            [else (set! times (+ times 1)) (f in)]))))

;; Validate
(define s (make-monitored (lambda (n) (+ n 1))))
(s 1)
(s 2)
(s 'how-many-calls?)
(s 'reset-count)
(s 'how-many-calls?)

0

### Exercise 3.3

Modify the `make-account` procedure so that
it creates password-protected accounts. That is, `make-account` should take a symbol as an additional argument, as in

```scheme
(define acc (make-account 100 'secret-password))
```

The resulting account object should process a request only if it is accompanied by the password with which the account was created, and should otherwise return a complaint:

```scheme
((acc 'secret-password 'withdraw) 40) 60
((acc 'some-other-password 'deposit) 50) "Incorrect password"
```

----

In [19]:
(define (make-account balance password)
  (define (withdraw amount p)
    (cond [(not (eq? p password)) "Incorrect password"]
          [(> amount balance) "Insufficient funds"]
          [else (set! balance (- balance amount)) balance]))

  (define (deposit amount p)
    (cond [(not (eq? p password)) "Incorrect password"]
          [else (set! balance (+ balance amount)) balance]))
  
  (define (dispatch p m)
    (cond [(eq? m 'withdraw) (lambda (n) (withdraw n p))]
          [(eq? m 'deposit) (lambda (n) (deposit n p))]
          [else (error "Unknown request: MAKE-ACCOUNT" m)]))

  dispatch)

;; Validate
(define acc (make-account 100 'secret-password))
((acc 'secret-password 'withdraw) 40) 60
((acc 'some-other-password 'deposit) 50)

"Incorrect password"

### Exercise 3.4

Modify the make-account procedure of [Exercise 3.3](#Exercise-3.3) by adding another local state variable so that, if an account is accessed more than seven consecutive times with an incorrect password, it invokes the procedure `call-the-cops`.

----

In [29]:
(define (make-account balance password)
  (let ([times 0])
    (define (validate p f)
       (cond [(eq? p password) (set! times 0) f]
             [(> times 6) 'call-the-cops]
             [else (set! times (+ times 1)) "Incorrect password"]))
    
    (define (withdraw amount)
      (cond [(> amount balance) "Insufficient funds"]
            [else (set! balance (- balance amount)) balance]))
    
    (define (deposit amount)
      (set! balance (+ balance amount)) balance)
    
    (define (dispatch p m)
      (cond [(eq? m 'withdraw) (lambda (n) (validate p (lambda () (withdraw n))))]
            [(eq? m 'deposit) (lambda (n) (validate p (lambda () (deposit n))))]
            [else (error "Unknown request: MAKE-ACCOUNT" m)]))
    dispatch))

;; Validate
(define acc (make-account 100 'secret-password))
((acc 'some-other-password 'deposit) 50)
((acc 'secret-password 'withdraw) 40) 60
((acc 'some-other-password 'deposit) 50)
((acc 'some-other-password 'deposit) 50)
((acc 'some-other-password 'deposit) 50)
((acc 'some-other-password 'deposit) 50)
((acc 'some-other-password 'deposit) 50)
((acc 'some-other-password 'deposit) 50)
((acc 'some-other-password 'deposit) 50)
((acc 'some-other-password 'deposit) 50)

'call-the-cops

## 3.1.2 The Benefits of Introducing Assignment